In [13]:
from langchain.tools import tool
import os
import dotenv

dotenv.load_dotenv()

os.getenv("STRAVA_ACCESS_TOKEN")

'e411ed63d06c048eea79f66a208d341e07af7434'

In [16]:
from stravalib.client import Client

client = Client()
client.access_token = os.getenv("STRAVA_ACCESS_TOKEN")
print(client.access_token)
route = client.get_route(3419324399883255614)
print(route)

e411ed63d06c048eea79f66a208d341e07af7434
bound_client=<stravalib.client.Client object at 0x000001E673FA7BB0> athlete=SummaryAthlete(bound_client=None, id=28331403, city='', country=None, created_at=datetime.datetime(2018, 2, 16, 22, 2, tzinfo=TzInfo(0)), firstname='Ciaran', lastname='Cooney', premium=True, profile='https://dgalywyr863hv.cloudfront.net/pictures/athletes/28331403/8502461/2/large.jpg', profile_medium='https://dgalywyr863hv.cloudfront.net/pictures/athletes/28331403/8502461/2/medium.jpg', resource_state=2, sex='M', state='', summit=True, updated_at=datetime.datetime(2025, 6, 27, 19, 10, 45, tzinfo=TzInfo(0))) created_at=datetime.datetime(2025, 10, 31, 14, 20, 15, tzinfo=TzInfo(0)) description='' distance=41531.91468578476 elevation_gain=836.2201419880938 estimated_moving_time=7294 id=3419324399883255614 id_str='3419324399883255614' map=Map(id='r3419324399883255614', polyline='_`x{FcwfLj@n@YlFDj@bB~FBl@Pv@P\\XZDVTlDj@nFj@zCr@nC~@rC^j@xI~PpBhDu@p@nCrGfAwAH[b@v@pAwAnCkCrOl]pDf

In [15]:
for r in client.get_athlete_routes():
    print(f"Name: {r.name}, ID: {r.id}, Distance: {r.distance/1000:.1f} km")

AttributeError: 'Client' object has no attribute 'get_athlete_routes'

In [18]:
from langchain.tools import BaseTool
import requests
from geopy.geocoders import Nominatim

In [26]:
city = "San Francisco, CA"
lat = None
lon = None
geolocator = Nominatim(user_agent="cycling_agent")
if city and (lat is None or lon is None):
    location = geolocator.geocode(city)
    if not location:
        pass #return [f"Could not find coordinates for city: {city}"]
    lat, lon = location.latitude, location.longitude

radius_km = 10  # 10 km radius
delta = radius_km / 111  # ~1 deg latitude ~ 111 km
bounds = f"{lat - delta},{lon - delta},{lat + delta},{lon + delta}"

url = "https://www.strava.com/api/v3/routes/explore"
params = {"bounds": bounds, "activity_type": "ride"}

In [31]:
headers = {"Authorization": f"Bearer {os.getenv('STRAVA_ACCESS_TOKEN')}"}

response = requests.get(url, headers=headers, params=params)
response.raise_for_status()

HTTPError: 404 Client Error: Not Found for url: https://www.strava.com/api/v3/routes/explore?bounds=37.68916870990991%2C-122.50941869009009%2C37.869348890090095%2C-122.32923850990991&activity_type=ride

In [34]:
client = Client()
client.access_token = os.getenv("STRAVA_ACCESS_TOKEN")
routes = client.get_athlete_routes(limit=10)

AttributeError: 'Client' object has no attribute 'get_athlete_routes'

In [35]:
import requests

ACCESS_TOKEN = os.getenv("STRAVA_ACCESS_TOKEN")

def get_my_routes(limit=10):
    url = "https://www.strava.com/api/v3/athlete/routes"
    headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}
    params = {"per_page": limit}
    
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()  # will raise HTTPError if unauthorized
    return response.json()

# Example usage
routes = get_my_routes(limit=5)
for r in routes:
    print(f"Name: {r['name']}, ID: {r['id']}, Distance: {r['distance']/1000:.1f} km, Elevation: {r['elevation_gain']} m")

Name: 31-10, ID: 3419324399883255614, Distance: 41.5 km, Elevation: 836.2201419880938 m
Name: 26.3 km Barcelona Ride Route, ID: 3418617717883952344, Distance: 26.3 km, Elevation: 361.28498287198727 m
Name: 152.9 km Sant Celoni Barcelona , ID: 3416539029638598966, Distance: 152.9 km, Elevation: 2807.514977174167 m
Name: 20.9 km Fuilla Ride Route, ID: 3410196991556211492, Distance: 20.9 km, Elevation: 1300.9496615819403 m
Name: 24.1 km Salses-le-Château Ride Route, ID: 3409846548871571538, Distance: 24.2 km, Elevation: 85.06885409105134 m


In [ ]:
from langchain.tools import BaseTool
import requests

class UserStravaRoutesTool(BaseTool):
    """Tool to get user's Strava routes. Currently requires valid access token and only enables access to athletes routes."""
    name = "user_strava_routes"
    description = (
        "Get a list of the user's Strava routes. Returns the name, ID, distance (in km), and elevation gain (in meters) of each route. "
        "Requires a valid Strava access token. "
    )

    def __init__(self, max_routes: int = 5):
        self.access_token = os.getenv("STRAVA_ACCESS_TOKEN")
        self.headers = {"Authorization": f"Bearer {self.access_token}"}
        self.max_routes = max_routes
        self.url = "https://www.strava.com/api/v3/athlete/routes"

    def _run(self) -> List[dict]:
        params = {"per_page": self.max_routes}
        response = requests.get(self.url, headers=self.headers, params=params)
        response.raise_for_status()
        routes = response.json()
        if not routes:
            return "No routes found for the user."
        result = []
        for r in routes:
            result.append({
                "name": r["name"],
                "id": r["id"],
                "distance_km": f"{r['distance']/1000:.1f} km",
                "elevation_m": f"{r['elevation_gain']} m"
            })
        return result
